```{contents}
```
## Callback Handlers

**Callback handlers** are hooks that let you **observe, log, stream, or react to internal events** occurring during runnable execution—such as chain start/end, LLM token generation, tool calls, and errors.

They are a core observability and control mechanism in LangChain.

```
Runnable Execution
   ├── on_chain_start
   ├── on_llm_start
   ├── on_llm_new_token
   ├── on_tool_start
   ├── on_chain_end / on_error
```

---

### Why Callback Handlers Are Needed

Callback handlers enable:

* Token streaming to UI
* Debugging and tracing
* Logging prompts and responses
* Metrics and monitoring
* Guardrails and policy checks

Without callbacks, runnable execution is a **black box**.

---

### Architecture View

![Image](https://miro.medium.com/1%2AL-qolcgDJOGwxdZ9QaM1vA.jpeg)

![Image](https://masterofcode.com/wp-content/uploads/2023/11/LOFTs-Architecture.png)

![Image](https://framerusercontent.com/images/8WL6CwrJPxTeDBwqoQ4CNBJpMMc.jpg?height=640\&width=1600)

---

### Callback Lifecycle (High Level)

1. Runnable starts
2. Callback handler is notified
3. Sub-steps emit events
4. Handler processes events
5. Runnable finishes or errors

Callbacks **do not change outputs** by default—they **observe**.

---



### Built-in Callback Handler (StdOut)



In [1]:
from langchain_classic.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm.invoke("Explain callback handlers briefly")


Callback handlers are functions that are passed as arguments to other functions and are invoked by those functions at a later time. They are commonly used in event-driven programming to handle asynchronous tasks and respond to events. Callback handlers allow for modular and flexible code as they enable functions to be executed in response to specific events or conditions.

AIMessage(content='Callback handlers are functions that are passed as arguments to other functions and are invoked by those functions at a later time. They are commonly used in event-driven programming to handle asynchronous tasks and respond to events. Callback handlers allow for modular and flexible code as they enable functions to be executed in response to specific events or conditions.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--019b4c29-589f-7bd1-a341-c4da5e645fec', usage_metadata={'input_tokens': 12, 'output_tokens': 63, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



**What happens**

* Tokens are printed to stdout as they arrive
* No custom code required

---

### Custom Callback Handler (Most Important)

#### Define a Custom Handler



In [4]:
from langchain_classic.callbacks.base import BaseCallbackHandler

class MyCallbackHandler(BaseCallbackHandler):
    def on_chain_start(self, serialized, inputs, **kwargs):
        print("Chain started:", inputs)

    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)

    def on_chain_end(self, outputs, **kwargs):
        print("\nChain ended:", outputs)




---

#### Attach It to a Runnable



In [5]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Explain {topic} in one sentence"
)

llm = ChatOpenAI(
    streaming=True,
    callbacks=[MyCallbackHandler()]
)

chain = prompt | llm

chain.invoke({"topic": "callback handlers"})

Callback handlers are functions that are passed as arguments to other functions and are called when a certain event or condition occurs.

AIMessage(content='Callback handlers are functions that are passed as arguments to other functions and are called when a certain event or condition occurs.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--019b4c2b-55ca-7293-bca1-04335331b79c', usage_metadata={'input_tokens': 14, 'output_tokens': 23, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



---

### What Events You Can Hook Into

Common callback methods:

```text
on_chain_start / on_chain_end
on_llm_start / on_llm_end
on_llm_new_token
on_tool_start / on_tool_end
on_retriever_start / on_retriever_end
on_error
```

You implement **only what you need**.

---

### Callback Handlers with RunnableSequence



In [6]:
from langchain_core.runnables import RunnableLambda

chain = (
    RunnableLambda(lambda x: x.upper())
    | llm
)

chain.invoke(
    "callbacks in sequence",
    config={"callbacks": [MyCallbackHandler()]}
)


Chain started: callbacks in sequence
Chain started: callbacks in sequence

Chain ended: CALLBACKS IN SEQUENCE
11.. The The first first callback callback function function is is executed executed.
.
22.. Once Once the the first first callback callback function function completes completes its its task task,, the the next next callback callback function function in in the the sequence sequence is is invoked invoked.
.
33.. This This process process continues continues until until all all the the callback callback functions functions in in the the sequence sequence have have been been executed executed..
Chain ended: content='1. The first callback function is executed.\n2. Once the first callback function completes its task, the next callback function in the sequence is invoked.\n3. This process continues until all the callback functions in the sequence have been executed.' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier':

AIMessage(content='1. The first callback function is executed.\n2. Once the first callback function completes its task, the next callback function in the sequence is invoked.\n3. This process continues until all the callback functions in the sequence have been executed.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--019b4c2b-d458-77d3-8aea-ddfc766e3f75', usage_metadata={'input_tokens': 13, 'output_tokens': 47, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



Callbacks receive events from **all steps** in the sequence.

---

### Per-Invocation vs Global Callbacks

#### Per-invocation



In [7]:
chain.invoke(
    "Explain callbacks",
    config={"callbacks": [MyCallbackHandler()]}
)

Chain started: Explain callbacks
Chain started: Explain callbacks

Chain ended: EXPLAIN CALLBACKS
CallbacksCallbacks are are functions functions that that are are passed passed as as arguments arguments to to another another function function and and are are intended intended to to be be executed executed once once the the main main function function has has completed completed its its task task.. They They are are a a way way to to handle handle asynchronous asynchronous operations operations in in JavaScript JavaScript,, such such as as fetching fetching data data from from an an API API or or executing executing a a time time-consuming-consuming computation computation.

.

WhenWhen a a function function accepts accepts a a callback callback as as an an argument argument,, it it can can call call that that callback callback once once it it has has completed completed its its task task.. This This allows allows the the original original function function to to continue continue runni

AIMessage(content='Callbacks are functions that are passed as arguments to another function and are intended to be executed once the main function has completed its task. They are a way to handle asynchronous operations in JavaScript, such as fetching data from an API or executing a time-consuming computation.\n\nWhen a function accepts a callback as an argument, it can call that callback once it has completed its task. This allows the original function to continue running other tasks while waiting for the callback to be executed.\n\nCallbacks are commonly used in event handlers, AJAX requests, and other asynchronous operations in JavaScript. They are a key concept in modern web development, as they allow for non-blocking code execution and help improve the performance of web applications.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--019b4c2c-261a-71b1-8342-a75


#### Global (applies everywhere)

```python
llm = ChatOpenAI(callbacks=[MyCallbackHandler()])
```

---

### Callbacks + Streaming (Real-Time UI)

Callback handlers are the **foundation** of:

* Token streaming
* SSE / WebSocket updates
* Progress indicators

Example usage:

```python
def on_llm_new_token(self, token, **kwargs):
    websocket.send_text(token)
```

---

### Callbacks in Async Execution

```python
async for chunk in llm.astream(
    "Explain async callbacks",
    config={"callbacks": [MyCallbackHandler()]}
):
    pass
```

Async and sync callbacks work the same conceptually.

---

### Callback Handlers vs Middleware

| Aspect      | Callback Handler | Middleware    |
| ----------- | ---------------- | ------------- |
| Purpose     | Observe events   | Modify flow   |
| Granularity | Step-level       | Request-level |
| Streaming   | Yes              | No            |
| LCEL-native | Yes              | No            |

---

### Common Use Cases

* Streaming tokens to UI
* Logging prompts/responses
* Cost and token accounting
* Tracing (LangSmith)
* Policy checks and audits
* Debugging chains and agents

---

### Common Mistakes

* Doing heavy blocking work in callbacks
* Logging sensitive data
* Forgetting callbacks don’t alter output
* Attaching callbacks at the wrong scope

---

### Mental Model

Callback handlers are **event listeners** for runnable execution.

```
Runnable runs → events fire → handlers react
```

---

### Key Takeaways

* Callback handlers provide **observability**
* They hook into every stage of execution
* Essential for streaming, logging, and monitoring
* Core building block for production LLM systems